<a href="https://colab.research.google.com/github/espada105/NewRecipe/blob/main/Llama3_2_11B_Vision_Instruct_NewRecipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#라이브러리 설치

In [1]:
!pip install -U transformers
!pip install transformers datasets accelerate
!pip install llama-stack

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.1 MB 11.2 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/25.1 MB 10.1 MB/s eta 0:00:03
   ---------- ----------------------------- 6.3/25.1 MB 10.2 MB/s eta 0:00:02
   ------------- -------------------------- 8.4/25.1 MB 10.4 MB/s eta 0:00:02
   ----------------- ---------------------- 10.7/25.1 MB 10.3 MB/s eta 0:00:02
   -------------------- ------------------- 13.1/25.1 MB 10.5 MB/s eta 0:00:02
   ------------------------ --------------- 15.5/25.1 MB 10.8 MB/s eta 0:

In [2]:
import torch
from transformers import VisionEncoderDecoderModel, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
from PIL import Image

c:\Users\PC\Documents\GitHub\ComputerVision-and-Deep-learning\NewRecipe\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#디바이스 설정

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(device)

cpu


#허깅페이스 로그인

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

#모델 및 프로세서 로드

In [6]:
# 8비트 양자화 설정
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# 모델과 프로세서를 8비트 양자화로 로드
processor = AutoProcessor.from_pretrained("meta-llama/Llama-3.2-11B-Vision-Instruct")
model = AutoModelForImageTextToText.from_pretrained(
    "meta-llama/Llama-3.2-11B-Vision-Instruct",
    quantization_config=quantization_config,
    device_map="auto"  # 자동 장치 매핑 사용
)

ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

#모델 불러오기

#레시피 데이터셋 로드

In [8]:
data_dir = "/content/drive/MyDrive/NewRecipe/full_dataset.csv"

recipe_dataset = load_dataset("csv", data_files=data_dir)

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
print(recipe_dataset['train'].select(range(5)))  # 처음 5개 행 출력 (데이터 로드 확인용임)


Dataset({
    features: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source', 'NER'],
    num_rows: 5
})


In [15]:
# 레시피 데이터셋에서 필수 정보 확인 (재료와 레시피 이름)
def get_relevant_recipes(ingredients):
    relevant_recipes = []

    for recipe in recipe_dataset["train"]:
        # 레시피에서 재료 정보 가져오기
        recipe_ingredients = recipe.get("ingredients", "").lower()

        # 모든 입력된 재료가 레시피에 포함되는지 확인
        if all(ingredient in recipe_ingredients for ingredient in ingredients):
            relevant_recipes.append({
                "title": recipe.get("title", "No Title"),              # 레시피 제목
                "ingredients": recipe_ingredients,                      # 레시피 재료
                "instructions": recipe.get("instructions", "No Instructions")  # 레시피 조리법
            })

    return relevant_recipes


#이미지에서 식재료 객체 인식

In [58]:

def recognize_ingredients(image_path):
    # 이미지 열기 및 RGB로 변환
    image = Image.open(image_path).convert("RGB")

    # 이미지 전처리 후 텐서를 'device'로 이동
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}  # 모든 입력을 동일한 'device'로 이동

    # 모델 출력 생성 (max_new_tokens 설정 추가)
    outputs = model.generate(**inputs, max_new_tokens=50)  # max_new_tokens로 출력 길이 조정

    # 결과 디코딩 (디코딩 전에 CPU로 이동)
    ingredients = processor.tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)  # .tokenizer 사용
    ingredients_list = ingredients.lower().split(", ")

    return ingredients_list


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 39.56 GiB of which 834.81 MiB is free. Process 60209 has 38.74 GiB memory in use. Of the allocated memory 37.99 GiB is allocated by PyTorch, and 268.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# 이미지로부터 식재료 인식 및 레시피 추천

In [56]:
def get_recipe_recommendations(image_path):
    # 이미지에서 식재료 인식
    recognized_ingredients = recognize_ingredients(image_path)
    print(f"Recognized ingredients: {recognized_ingredients}")

    # 인식된 식재료를 바탕으로 레시피 검색
    recipes = get_relevant_recipes(recognized_ingredients)
    if recipes:
        print("Recommended Recipes:")
        for i, recipe in enumerate(recipes[:5]):  # 상위 5개 레시피 추천
            print(f"\nRecipe {i+1}: {recipe['title']}")
            print(f"Ingredients: {recipe['ingredients']}")
            print(f"Instructions: {recipe['instructions']}")
    else:
        print("No recipes found for the given ingredients.")




# 예제 실행

In [57]:
image_path = "/content/drive/MyDrive/NewRecipe/무.jpg"  # 입력할 이미지 경로 -> 이건 우리가 테스트할 이미지 넣으면 됨 ㅇㅇ..
get_recipe_recommendations(image_path)
print("Recognized ingredients:", ingredients)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)